## Prepare

In [ ]:
import os
from pathlib import PurePath, Path
os.environ["MICROMAZEMASTER_ENV"] = "testing"
WORKING_DIR = PurePath(os.getcwd()).parent
os.chdir(WORKING_DIR)

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.rich import trange

from micromazemaster.utils.logging import logger
from micromazemaster.utils.tflite import convert_onnx_to_tflite, convert_tflite_to_header, convert_pytorch_to_onnx

logger.info(f"Current working directory: {WORKING_DIR}")

Path(WORKING_DIR/"local_data").mkdir(exist_ok=True)

## Train a model

In [ ]:
f = lambda x: torch.sin(x)

x = torch.linspace(0, 12.5, 128)


# Define the neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hidden1 = nn.Linear(1, 64)
        self.hidden2 = nn.Linear(64, 128)
        self.output = nn.Linear(128, 1)

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = self.output(x)
        return x


model_fp32 = Net()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_fp32.parameters(), lr=0.01)  # Using Adam optimizer

for epoch in trange(100000, desc="Training"):
    running_loss = 0.0
    optimizer.zero_grad()
    outputs = model_fp32(x.unsqueeze(1))
    loss = criterion(outputs.squeeze(), f(x))
    loss.backward()
    optimizer.step()
    running_loss += loss.item()

    if epoch % 10000 == 0:
        logger.info("Epoch {}: Loss = {}".format(epoch, loss.detach().numpy()))


# Plot the actual function and predicted function
actual_y = torch.tensor([f(p) for p in x])
predicted_y = model_fp32(x.unsqueeze(1)).squeeze()
plt.plot(x, actual_y, "g", label="Actual Function")
plt.plot(x, predicted_y.detach().numpy(), "b", label="Predicted Function")
plt.legend()
plt.show()

## Save the model to onnx

In [ ]:
# model has 1 input in 1 Dimension
torch_input = torch.randn(1, 1)

convert_pytorch_to_onnx(model_fp32, torch_input, "local_data/model.onnx")

## Convert from onnx to tflite format

In [ ]:
convert_onnx_to_tflite("local_data/model.onnx", "local_data/saved_model")

## Convert from tflite to header c-array for inference on Microcontroller

In [ ]:
convert_tflite_to_header("local_data/saved_model/model_float32.tflite", "local_data/model_float32.h")